In [7]:
import pandas as pd

df = pd.read_csv('data/change_df.csv')
df = df.drop(columns=['출생아수'])
df.head(), df.shape, df['지역별'].unique()


(  지역별    시점   저출산율
 0  서울  2000  1.275
 1  서울  2001  1.111
 2  서울  2002  1.006
 3  서울  2003  1.014
 4  서울  2004  1.015,
 (408, 3),
 array(['서울', '부산', '대구', '인천', '광주', '대전', '울산', '세종', '경기', '강원', '충북',
        '충남', '전북', '전남', '경북', '경남', '제주'], dtype=object))

In [8]:
# '시점'을 datetime 형식으로 변환
df['시점'] = pd.to_datetime(df['시점'].astype(str) + '-01-01')

# 인덱스로 설정
yearly_data = df.set_index('시점')

# '저출산율'을 숫자형으로 변환 (문자열에서 숫자로)
yearly_data['저출산율'] = pd.to_numeric(yearly_data['저출산율'], errors='coerce')

# 지역별로 그룹화 후 리샘플링
monthly_dfs = []

for region, group in yearly_data.groupby('지역별'):
    monthly_df = group.resample('ME').agg({'저출산율': 'mean'}).round(3)
    
    # 보간
    monthly_df = monthly_df.interpolate(method='linear')
    
    # NaN 값 처리
    monthly_df.fillna(0, inplace=True)
    
    # 시점 다시 설정
    monthly_df.reset_index(inplace=True)
    
    # '시점'이 datetime인지 확인 후, 형식 변경
    if pd.api.types.is_datetime64_any_dtype(monthly_df['시점']):
        monthly_df['시점'] = monthly_df['시점'].dt.strftime('%Y.%m')
    
    # 지역별 컬럼 추가
    monthly_df['지역별'] = region
    
    monthly_dfs.append(monthly_df)

# 모든 지역별 데이터를 하나의 DataFrame으로 결합
final_monthly_df = pd.concat(monthly_dfs, ignore_index=True)
final_monthly_df['저출산율'] = final_monthly_df['저출산율'].map(lambda x: f"{x:.2f}")

# 결과 확인
print(final_monthly_df.head(12), final_monthly_df.shape, final_monthly_df['지역별'].unique())

# 데이터 저장
final_monthly_df.to_csv('data/저출산율(월별).csv', index=False)

         시점  저출산율 지역별
0   2000.01  1.60  강원
1   2000.02  1.58  강원
2   2000.03  1.57  강원
3   2000.04  1.55  강원
4   2000.05  1.54  강원
5   2000.06  1.52  강원
6   2000.07  1.51  강원
7   2000.08  1.49  강원
8   2000.09  1.48  강원
9   2000.10  1.46  강원
10  2000.11  1.44  강원
11  2000.12  1.43  강원 (4709, 3) ['강원' '경기' '경남' '경북' '광주' '대구' '대전' '부산' '서울' '세종' '울산' '인천' '전남' '전북'
 '제주' '충남' '충북']
